In [1]:
from typing import Any, Tuple, Optional
import numpy as np
import pandas as pd
from scipy.stats import nbinom, gamma, poisson, hypergeom
import scipy.integrate as integrate
import copy
import multiprocessing
import uuid
from math import exp, log
from tqdm import tqdm
from itertools import product
import collections

In [ ]:
# Based on original TADA software (He et al. 2013) and rewrite (Klei 2015)

# Set seed for reproducibility
np.random.seed(100)

def TADA(tada_counts, sample_counts, mu, hyperpar, denovo_only, mu_frac=1, pi_gene=1):
    print("CALCULATION OF TADA TEST STATISTICS")
    print("checking the input for consistent variable names")

    mutation_types = list(tada_counts.keys())
    n_mutation = len(mutation_types)
    if not isinstance(mu_frac, pd.DataFrame):
        mu_frac = pd.DataFrame([[mu_frac] * n_mutation], columns=mutation_types)

    if not isinstance(pi_gene, pd.DataFrame):
        pi_gene = pd.DataFrame([[pi_gene] * len(tada_counts)], columns=mutation_types)

    if (
        sum([mutation_type in mu for mutation_type in mutation_types]) != n_mutation
        or sum([mutation_type in mu_frac for mutation_type in mutation_types]) != n_mutation
        or sum([mutation_type in hyperpar for mutation_type in mutation_types]) != n_mutation
        or sum([mutation_type in pi_gene for mutation_type in mutation_types]) != n_mutation
        or sum([mutation_type in denovo_only for mutation_type in mutation_types]) != n_mutation
        or sum([mutation_type in sample_counts for mutation_type in mutation_types]) != n_mutation
    ):
        return "mismatch in names for the different variables"

    names_trans_categories = list(sample_counts[mutation_types[0]].keys())
    names_N = ["dn", "ca", "cn"]
    for mutation_type in mutation_types:
        if sum(name_N in tada_counts[mutation_type] for name_N in names_N) != 3:
            return f"columns of {mutation_type} do not match the required 'dn', 'ca', 'cn'"

    n_gene = len(mu)
    n_mutation = len(mutation_types)

    BF = pd.DataFrame()
    for mutation_type in mutation_types:
        print(f"working on :: {mutation_type}")
        BF_mut = pd.DataFrame(
            np.zeros((len(tada_counts[mutation_type]), 1)),
            columns=["BF"]
        )
        for i in range(len(tada_counts[mutation_type])):
            counts = tada_counts[mutation_type].iloc[i]
            n = sample_counts[mutation_type].iloc[i]
            print(tada_counts[mutation_type].iloc[i])
            print([names_N[i]])
            print(calculate_BF(
                counts,
                n,
                mu[mutation_type],
                mu_frac[mutation_type],
                hyperpar[mutation_type],
                denovo_only[mutation_type],
                pi_gene[mutation_type],
            ))
            BF_mut.iloc[i] = calculate_BF(
                counts,
                n,
                mu[mutation_type],
                mu_frac[mutation_type],
                hyperpar[mutation_type],
                denovo_only[mutation_type],
                pi_gene[mutation_type],
            )
        BF[mutation_type] = BF_mut["BF"]

    BF.columns = mutation_types
    BF.index = tada_counts[mutation_types[0]].index

    BF_total = np.exp(np.log(BF).sum(axis=1))
    return {"BF": BF, "BF_total": BF_total}


def TADAnull(
    tada_counts,
    sample_counts,
    mu,
    hyperpar,
    denovo_only,
    mu_frac=1,
    n_rep=100,
    dn_max=20,
    ca_max=200,
    cn_max=200,
    max_gap=50,
):
    print("CALCULATION OF TADA TEST STATISTICS UNDER THE NULL HYPOTHESIS")
    mutations = list(hyperpar.keys())
    if not isinstance(mu_frac, pd.DataFrame):
        mu_frac = pd.DataFrame([mu_frac] * len(mutations), columns=mutations)

    table_BF_dn = {}
    for mutation in mutations:
        print(f"working on creating DN table for :: {mutation}")
        x = np.arange(dn_max + 1)
        param = hyperpar[mutation]
        n = sample_counts[mutation]["dn"]

        BF = table_BF_dn_wrapper(x, n, mu[mutation] * mu_frac[mutation], param["gamma.mean.dn"], param["beta.dn"])
        table_BF_dn[mutation] = pd.DataFrame(BF, columns=[f"X{value}" for value in x])

    table_BF_cc = {}
    for mutation in mutations:
        if not denovo_only[mutation]:
            print(f"working on creating CC table for :: {mutation}")
            tada_counts[mutation]["Ncc"] = tada_counts[mutation][["ca", "cn"]].sum(axis=1)
            Ncc = sorted(tada_counts[mutation]["Ncc"])
            Ncc_gaps = np.diff(Ncc)
            i_gap = np.argmax(Ncc_gaps > max_gap)
            n_ca = min(ca_max, Ncc[i_gap])
            n_cn = min(cn_max, Ncc[i_gap])
            x = pd.DataFrame(list(product(range(n_ca + 1), range(n_cn + 1))), columns=["ca", "cn"])
            param = hyperpar[mutation]
            n = sample_counts[mutation][["ca", "cn"]]

            BF = table_BF_cc_wrapper(
                x,
                n,
                param["gamma.mean.CC"],
                param["beta.CC"],
                param["rho1"],
                param["nu1"],
                param["rho0"],
                param["nu0"],
            )
            table_BF_cc[mutation] = BF.values.reshape(n_ca + 1, n_cn + 1)

    return {"table_BF_dn": table_BF_dn, "table_BF_cc": table_BF_cc}


def calculate_BF(i_gene, counts, n, mu, mu_frac, hyperpar, denovo_only, pi_gene):
    # Wrapper so that lapply can be used to determine the BF for a gene and a particular mutation variant
    # i_gene: gene of interest
    # counts: counts for a particular variant, dataframe with vectors for dn, ca, cn
    # n: total samples counts, dataframe with entries for dn, ca, cn
    # mu: vector with mutation rates for the variant of interest for each gene
    # mu_frac: fraction to multiply mu with for the variant of interest
    # hyperpar: dataframe with entries for gamma.mean.dn, beta.dn, gamma.mean.CC, beta.CC, rho1, nu1, rho0, nu0 for the
    #           variant of interest
    # denovo_only: Boolean vector indicating whether only denovo contribution (denovo_only = True) or a combination of
    #              denovo and case-control contributions is to be used (denovo_only=False)
    # pi_gene: vector with K vectors of estimated fractions of causal variants, one for each class of variants.
    #          These fractions will be used to set gene-specific RR (case-control)

    if i_gene % 100 == 0 or i_gene == counts.shape[0]:
        pbar = tqdm(total=counts.shape[0])

    ### set the hyper parameters for this gene
    hyperpar_gene = hyperpar.copy()
    RR_product = hyperpar_gene['gamma.mean.CC'] * hyperpar_gene['beta.CC']
    hyperpar_gene['gamma.mean.CC'] = hyperpar_gene['gamma.mean.CC'] * pi_gene[i_gene] + (1 - pi_gene[i_gene])
    hyperpar_gene['beta.CC'] = RR_product / hyperpar_gene['gamma.mean.CC']

    # Determine the BAYES factor
    BF = bayes_factor(x=counts.iloc[i_gene, :], n=n, mu=mu[i_gene] * mu_frac, param=hyperpar_gene, denovo_only=denovo_only)

    if i_gene % 100 == 0 or i_gene == counts.shape[0]:
        pbar.update(1)
        pbar.close()

    return BF

def bayes_factor(x, n, mu, param, denovo_only):
    # Bayes factor of the gene combining de novo and case-control
    # x: a list of (dn, ca, cn), counts in de novo, cases and controls
    # n: a list of (dn, ca, cn), sample sizes
    # param: (gamma.mean.dn, beta.dn, gamma.mean.CC, beta.CC, rho1, nu1, rho0, nu0)
    # denovo_only: Boolean indicating whether only denovo contribution (True) or a combination of
    #              denovo and case-control contributions is to be used (False)
    # Prior distribution of RR in de novo: gamma.dist.dn ~ Gamma(gamma.mean.dn * beta.dn, beta.dn)
    # Prior distribution of RR in C/C data: gamma.dist.cc ~ Gamma(gamma.mean.CC * beta.CC, beta.CC)
    # Prior distribution of q|H1: Gamma(rho1, nu1)
    # Prior distribution of q|H0: Gamma(rho0, nu0)

    # Contribution of denovo variants in families
    BF_dn = bayes_factor_dn(x_dn=x['dn'], n_dn=n['dn'], mu=mu, gamma_dn=param['gamma.mean.dn'], beta_dn=param['beta.dn'])
    if not denovo_only:
        # Contribution of variants in cases and controls
        BF_cc = bayes_factor_cc(x_cc=x[['ca', 'cn']], n_cc=n[['ca', 'cn']], gamma_cc=param['gamma.mean.CC'], beta_cc=param['beta.CC'],
                                rho1=param['rho1'], nu1=param['nu1'], rho0=param['rho0'], nu0=param['nu0'])
    else:
        BF_cc = 1
    # Combine the pieces of information
    BF = BF_dn * BF_cc
    return BF

def bayes_factor_dn(x_dn, n_dn, mu, gamma_dn, beta_dn):
    # Bayes factor of de novo counts of a gene
    # x_dn: the de novo count
    # n_dn: the sample size (number of families)
    # mu: the mutation rate (of this type of mutational events)
    # Prior distribution of RR: gamma ~ Gamma(gamma_dn * beta_dn, beta_dn)
    marg_lik0 = poisson.pmf(x_dn, 2 * n_dn * mu)
    marg_lik1 = nbinom.pmf(x_dn, gamma_dn * beta_dn, beta_dn / (beta_dn + 2 * n_dn * mu))
    BF = marg_lik1 / marg_lik0
    return BF


def bayes_factor_cc(x_cc, n_cc, gamma_cc, beta_cc, rho1, nu1, rho0, nu0):
    # Bayes factor of the case-control data
    # BF_cn and BF_ca: contribution from control and case data, respectively
    # Input: the count data x_cc, the sample size n_cc and the parameters gamma_cc, beta_cc, rho1 and nu1
    # Prior distribution of RR: gamma ~ Gamma(gamma_cc * beta_cc, beta_cc)
    # Prior distribution of q|H1: Gamma(rho1, nu1)
    # Prior distribution of q|H0: Gamma(rho0, nu0)
    marglik0_cc = evidence_null_cc(x_cc, n_cc, rho0, nu0)
    marglik1_cc = evidence_alt_cc(x_cc, n_cc, gamma_cc, beta_cc, rho1, nu1)
    BF_cn = marglik1_cc["cn"] / marglik0_cc["cn"]
    BF_ca = marglik1_cc["ca"] / marglik0_cc["ca"]
    BF = BF_cn * BF_ca
    return BF

def evidence_null_cc(x_cc, n_cc, rho0, nu0):
    # model evidence of case-control data: P(x_1,x_0|H_0)
    # Input: the count data x_cc, the sample size n_cc, and the rho0 and nu0
    # Prior distribution of q|H0: Gamma(rho0, nu0)
    marglik0_ctrl_log = np.log(nbinom.pmf(x_cc["cn"], rho0, nu0 / (nu0 + n_cc["cn"])))
    marglik0_case_log = np.log(nbinom.pmf(x_cc["ca"], rho0 + x_cc["cn"], (nu0 + n_cc["cn"]) / (nu0 + n_cc["cn"] + n_cc["ca"])))
    marglik0_log = marglik0_ctrl_log + marglik0_case_log

    return {"cn": np.exp(marglik0_ctrl_log), "ca": np.exp(marglik0_case_log), "total": np.exp(marglik0_log)}

def evidence_alt_cc(x_cc, n_cc, gamma_cc, beta_cc, rho1, nu1, q_lower=1e-8, q_upper=0.1, debug=False):
    # model evidence of case-control data: P(x_1,x_0|H_1)
    # Input: the count data x_cc, the sample size n_cc, and the parameters gamma_cc, beta_cc, rho1, and nu1
    # Prior distribution of RR: gamma ~ Gamma(gamma_cc*beta_cc, beta_cc)
    # Prior distribution of q|H1: Gamma(rho1, nu1)
    def integrand(u):
        q = np.exp(u)
        return (nbinom.pmf(x_cc["ca"], gamma_cc * beta_cc, beta_cc / (beta_cc + n_cc["ca"] * q)) *
                gamma.pdf(q, rho1 + x_cc["cn"], scale=1 / (nu1 + n_cc["cn"])) * np.exp(u))

    marglik1_ctrl = nbinom.pmf(x_cc["cn"], rho1, nu1 / (nu1 + n_cc["cn"]))
    result = integrate.quad(integrand, np.log(q_lower), np.log(q_upper))
    marglik1_case = result[0]

    marglik1 = marglik1_ctrl * marglik1_case

    return {"cn": marglik1_ctrl, "ca": marglik1_case, "total": marglik1}
def table_BF_dn_wrapper(i_x, x, n_dn, mu, gamma_dn, beta_dn):
    # a wrapper function used for generating the denovo table, see bayes.factor.dn for a description of the variables
    BF = bayes_factor_dn(x[i_x], n_dn=n_dn, mu=mu, gamma_dn=gamma_dn, beta_dn=beta_dn)
    return BF


def table_BF_cc_wrapper(i_x, x, n_cc, gamma_cc, beta_cc, rho1, nu1, rho0, nu0):
    # a wrapper function used for generating the case-control table, see bayes.factor.cc for a description of the variables
    if (i_x % 100 == 0 or i_x == len(x)) and len(x) > 1000:
        pb = tqdm(total=len(x))
        pb.update(i_x)
    BF = bayes_factor_cc(x[i_x], n_cc=n_cc, gamma_cc=gamma_cc, beta_cc=beta_cc, rho1=rho1, nu1=nu1, rho0=rho0, nu0=nu0)
    return BF

def permute_gene(i_gene, mu_rate, counts, n, nrep, param, denovo_only, table_cc, table_dn):
    # Compute permutation BFs of one gene
    # mu_rate: the mutation rate of a gene for the variant of interest
    # counts: dn, ca, and co counts for the variant of interest to be permuted, this also has a column for Ncc=ca+cn
    # n: sample size, for values for de novo, case, control, and case+control
    # nrep: number of permutations
    # param: set of hyper parameters for the variant of interest.
    # table_cc: table of precomputed BFs for case control events of size max.ca by max.cn for the variant of interest
    # table_dn: table of precomputed BFs for denovo events of size number of genes by max.dn for the variant of interest
    # Output: vector of nrep BF generated under the null hypothesis

    if i_gene % 100 == 0 or i_gene == counts.shape[0]:
        print(f"Progress: {i_gene}/{counts.shape[0]}")

    # generate permutation data for denovo events
    sample_dn = np.random.poisson(2 * nrep * mu_rate[i_gene], size=nrep)
    # look up the BF value in the table
    BF_dn = table_dn[i_gene, sample_dn]

    if not denovo_only:
        # when both denovo and case-control BF are needed
        # generate permutation data for case-control events
        max_ca = table_cc.shape[0]
        max_cn = table_cc.shape[1]
        sample_ca = np.zeros(nrep)
        sample_cn = np.zeros(nrep)

        for j in range(nrep):
            sample_ca[j] = np.random.hypergeometric(counts["Ncc"][i_gene], n["ca"] + n["cn"] - counts["Ncc"][i_gene], n["ca"])
            sample_cn[j] = counts["Ncc"][i_gene] - sample_ca[j]

        # find the generated counts that are outside of the pre-computed table
        i_na = np.where((sample_ca + 1 > max_ca) | (sample_cn + 1 > max_cn))[0]
        if len(i_na) > 0:
            # calculate their BF on a case by case basis
            BF_na = np.zeros(len(i_na))
            for idx, i in enumerate(i_na):
                BF_na[idx] = table_BF_cc_wrapper(data={"ca": sample_ca[i], "cn": sample_cn[i]},
                                                 n_cc=n[["ca", "cn"]],
                                                 gamma_cc=param["gamma.mean.CC"],
                                                 beta_cc=param["beta.CC"],
                                                 rho1=param["rho1"],
                                                 nu1=param["nu1"],
                                                 rho0=param["rho0"],
                                                 nu0=param["nu0"])

        # set the counts outside the range to missing
        sample_ca[sample_ca + 1 > max_ca] = np.nan
        sample_cn[sample_cn + 1 > max_cn] = np.nan

        # gather the BF values that can be taken from the pre-computed table
        BF_cc = table_cc[sample_ca + 1, sample_cn + 1]

        # replace the missing values with the pre-computed ones
        i_na = np.where(np.isnan(BF_cc))[0]
        if len(i_na) > 0:
            BF_cc[i_na] = BF_na

    else:
        # if denovo only needed then set BF_dn to 1
        BF_cc = np.ones(nrep)

    # determine the total BF from the two components
    BF = BF_cc * BF_dn

    return BF

def Bayesian_FDR(BF, pi0):
    # Bayesian FDR control (PMID:19822692, Section2.3)
    # BF: a vector of BFs
    # pi0: the prior probability that the null model is true
    # Return: the q-value of each BF, and the number of findings with q below alpha.

    # order the BF in decreasing order, need to retain order to get results back in proper order
    i_order = np.argsort(-BF)
    BF = BF[i_order]

    # convert BFs to PPA (posterior probability of alternative model)
    pi = 1 - pi0
    q = pi * BF / (1 - pi + pi * BF)  # PPA
    q0 = 1 - q  # posterior probability of null model

    # the FDR at each PPA cutoff
    FDR = np.cumsum(q0) / np.arange(1, len(BF) + 1)

    # reorder to the original order
    FDR[i_order] = FDR

    return FDR

def bayesFactor_pvalue(BF, BF_null):
    # determines the p-value for the BF using permutations under the null hypothesis BF_null
    # BF: vector with bayes factors based on the data
    # BF_null: vector with bayes factors based on permuted data

    BF_null = np.sort(BF_null)[::-1]
    pval = np.searchsorted(-BF_null, -BF) / len(BF_null)
    pval[pval == 0] = 0.5 / len(BF_null)

    return pval

def denovo_MOM(k, N, mu, C, beta, d=2, S=100, max_kvec=None):
    # Estimating relative risk and the number of multiple hits from de novo data
    # Input:  k - number of disease genes
    #         N - sample size
    #         mu - mutation rate for all genes
    #         C - observed number of de novo events
    #         beta - parameter of the prior distribution of gamma
    #         d - number of events to use (1 is 1 or more, 2 is 2 or more)
    #         S - number of samples to generate per gene
    #         max_kvec - used to generate a time line.
    # Output: gamma_mean - the average relative risk,
    #         M - the expected number of multi-hit genes

    if max_kvec is not None:
        if k % 100 == 0 or k == max_kvec:
            pb = tqdm(total=max_kvec)
            pb.update(k)

    m = len(mu)  # number of genes

    # enrichment of de novo events
    nu = C / (2 * N * np.sum(mu))

    # MOM estimator of gamma_mean
    gamma_mean = (nu - 1) * m / k + 1

    # expected M (choose d = 2)
    rs = count_multihit(N, mu, k / m, gamma_mean, beta, d=d, S=S)
    M = np.sum(rs['M1']) + np.sum(rs['M0'])

    return {'gamma.mean': gamma_mean, 'M': M}


def count_multihit(N, mu, pi, gamma_mean, beta, d, S):
    # Estimate the number of multihit genes in a genome.
    # N: sample size
    # mu: mutation rate for all genes
    # pi: ratio of number of risk genes and total number of genes
    # gamma_mean: the average relative risk
    # beta: parameter of the prior distribution of gamma
    # d: number of events to use (1 is 1 or more, 2 is 2 or more)
    # S: number of samples to generate per gene
    # Output: M0 - number of multiple-hit genes for the non-risk genes
    #         M1 - number of multiple-hit genes for risk genes

    m = len(mu)

    # M1: the number of causal genes having d or more de novo mutations
    p_alt = np.column_stack([multihit_prob(mu_i, N, gamma_mean, beta, d=d, S=S) for mu_i in mu])
    M1 = m * pi * np.mean(p_alt, axis=0)

    # M0: the number of non-causal genes having d or more de novo mutations
    p_null = np.column_stack([(1 - poisson.cdf(d_i, 2 * N * mu_i)) for mu_i in mu for d_i in d])
    p_null = p_null.reshape(m, len(d))
    M0 = m * (1 - pi) * np.mean(p_null, axis=0)

    result = pd.DataFrame({'d': d, 'M0': M0, 'M1': M1})
    return result

def multihit_prob(mu, N, gamma_mean, beta, d, S):
    # Prob. of having d or more de novo mutations under H1
    # Use simulation, but could also use analytic form
    # mu: mutation rate for a gene
    # N: sample size
    # gamma_mean: the average relative risk
    # beta: parameter of the prior distribution of gamma
    # d: number of events to use (1 is 1 or more, 2 is 2 or more)
    # S: number of samples to generate per gene
    # Output: p - average probability of having d or more de novo mutations

    gamma = gamma.rvs(gamma_mean * beta, scale=1 / beta, size=S)
    p = 1 - poisson.cdf(d, 2 * N * mu * gamma)
    return np.mean(p)

# Read mutation data
tada_file = "TADA_demo_counts_de-novo_and_inherited.txt"
tada_data = pd.read_table(tada_file)

# Specify the number of families and the number of cases and control samples included in the analysis
n_family = 4500
n_case = 1000
n_ctrl = 3000

data = {'dn': [n_family], 'ca': [n_case + n_family], 'cn': [n_ctrl + n_family]}
n = pd.DataFrame(data)
sample_counts = {'cls1': n, 'cls2': n}

# Create the mutational data used by TADA
cls1_counts = pd.DataFrame({'dn': tada_data['dn.cls1'],
                            'ca': tada_data['trans.cls1'] + tada_data['case.cls1'],
                            'cn': tada_data['ntrans.cls1'] + tada_data['ctrl.cls1']})
cls1_counts.index = tada_data['gene.id']

cls2_counts = pd.DataFrame({'dn': tada_data['dn.cls2'],
                            'ca': tada_data['trans.cls2'] + tada_data['case.cls2'],
                            'cn': tada_data['ntrans.cls2'] + tada_data['ctrl.cls2']})
cls2_counts.index = tada_data['gene.id']

tada_counts = {'cls1': cls1_counts, 'cls2': cls2_counts}

# Set up mutation rates
mu = pd.DataFrame({'cls1': tada_data['mut.cls1'], 'cls2': tada_data['mut.cls2']})

# Set up denovo only TRUE/FALSE, here we do not want to restrict ourselves to de novo only analyses
denovo_only = pd.DataFrame({'cls1': [False], 'cls2': [False]})

# Set up parameters
cls1_params = pd.DataFrame({'gamma.mean.dn': [20.0],
                            'beta.dn': [1],
                            'gamma.mean.CC': [2.3],
                            'beta.CC': [4.0],
                            'rho1': [0.1],
                            'nu1': [100],
                            'rho0': [0.1],
                            'nu0': [100]})

cls2_params = pd.DataFrame({'gamma.mean.dn': [4.7],
                            'beta.dn': [1],
                            'gamma.mean.CC': [1.0],
                            'beta.CC': [1000],
                            'rho1': [0.15],
                            'nu1': [100],
                            'rho0': [0.15],
                            'nu0': [100]})

hyperpar = {'cls1': cls1_params, 'cls2': cls2_params}
# Running TADA
re_TADA = TADA(tada_counts=tada_counts, sample_counts=sample_counts, mu=mu, hyperpar=hyperpar, denovo_only=denovo_only)

# Bayesian FDR control
re_TADA['qval'] = Bayesian_FDR(re_TADA['BF.total'], pi0=0.95)

# Run permutation to get the null distributions to use for calculating p-values for TADA
re_TADA_null = TADAnull(tada_counts=tada_counts, sample_counts=sample_counts, mu=mu, hyperpar=hyperpar, denovo_only=denovo_only, nrep=100)
re_TADA['pval'] = bayesFactor_pvalue(re_TADA['BF.total'], re_TADA_null['BFnull.total'])

# Top 10 genes based on BF.total
top_10_genes = re_TADA.sort_values(by='BF.total', ascending=False).head(10)